In [0]:
from pyspark.sql.functions import sum, min, max, col, when, concat_ws, collect_set, regexp_extract
import traceback

In [0]:
%run ../utility/util

In [0]:
def aggregate_invoice(df):
  '''
  Aggregate part sales based on root document number in invoice
  '''
  # Expression which derives onContract column value to be 1 if OpenDate is between ContractStart 
  # and ContractEnd, 0 otherwise
  on_contract = when(col('OpenDate').between(col('ContractStart'), col('ContractEnd')), 1).otherwise(0)
  df = df.groupBy('RootDocumentNumber') \
         .agg(sum('PartSales').alias('PartSales'), \
              min('OpenDate').alias('OpenDate'), \
              min('ContractStart').alias('ContractStart'), \
              max('ContractEnd').alias('ContractEnd')) \
         .withColumn('OnContract', on_contract)
  return df

In [0]:
def attribute_revenue(df):
  '''
  Merge document numbers with invoice data and calculate total part sales by meeting id
  '''
  # Expression to replace nulls with zero
  on_contract_null_to_zero = when(col('OnContract').isNull(), 0).otherwise(col('OnContract'))
  adf = aggregate_invoice(df)
  
  # specify the columns on which the tables are joined to avoid duplicate columns
  # spark collect_set method will remove nulls and duplicates
  df =  df.join(adf, ['RootDocumentNumber', 'PartSales', 'OpenDate', 'ContractStart', 'ContractEnd'], how='left') \
          .groupBy('MeetingId') \
          .agg(concat_ws(', ', collect_set('RootDocumentNumber')).alias('RootDocumentNumber'), \
               sum('PartSales').alias('PartSales'), \
               sum('OnContract').alias('OnContract')) \
          .withColumn('_OnContract', on_contract_null_to_zero).drop(col('OnContract')) \
          .withColumn('SalesType', regexp_extract(col('RootDocumentNumber'), '([CS])', 0))
  return df

In [0]:
def main():
  
  # Read data from Delta table
  try:
    parseddocumentnumber_df = table_reader_delta("appdb_equiplink", "partscallcentre_parseddocumentnumber")
    invoice_data_df = table_reader_delta("appdb_equiplink", "partscallcentre_invoice_data")
    joined_df = parseddocumentnumber_df.join(invoice_data_df, "RootDocumentNumber", how='left')
  except Exception as e:
    traceback.print_exc()
    raise Exception("Error while reading from Delta Table.")

  # Attributing revenue to calls
  try:
    result_df = attribute_revenue(df)
  except Exception as e:
    traceback.print_exc()
    print("Error while Attributing revenue to calls")
    return  
  
  # Store data to Delta table
  try:
    table_writer_delta(result_df, "appdb_equiplink", "partscallcentre_revenue", "overwrite")
    spark.catalog.refreshTable("appdb_equiplink.partscallcentre_revenue")
  except Exception as e:
    traceback.print_exc()
    raise Exception("Error while writing to Delta table appdb_equiplink.partscallcentre_revenue")

In [0]:
main()

Traceback (most recent call last):
 File "<command-842702219804308>", line 5, in main
 parseddocumentnumber_df = table_reader_delta("appdb_equiplink", "partscallcentre_parseddocumentnumber_test")
 File "<command-3014004920734363>", line 8, in table_reader_delta
 return spark.table(db_name+"."+table_name)
 File "/databricks/spark/python/pyspark/sql/session.py", line 722, in table
 return DataFrame(self._jsparkSession.table(tableName), self._wrapped)
 File "/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
 answer, self.gateway_client, self.target_id, self.name)
 File "/databricks/spark/python/pyspark/sql/utils.py", line 133, in deco
 raise_from(converted)
 File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: Table or view not found: appdb_equiplink.partscallcentre_parseddocumentnumber_test;;
'UnresolvedRelation [appdb_equiplink, partscallcentre_parseddocumentnumber_test]

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-842702219804308> in main () 
 4 try : 
 ----> 5 parseddocumentnumber_df = table_reader_delta ( "appdb_equiplink" , "partscallcentre_parseddocumentnumber_test" ) 
 6 invoice_data_df = table_reader_delta ( "appdb_equiplink" , "partscallcentre_invoice_data" ) 

 <command-3014004920734363> in table_reader_delta (db_name, table_name, columns) 
 7 else : 
 ----> 8 return spark . table ( db_name + "." + table_name ) 

 /databricks/spark/python/pyspark/sql/session.py in table (self, tableName) 
 721 """
 --> 722 return DataFrame ( self . _jsparkSession . table ( tableName ) , self . _wrapped ) 
 723 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Table or view not found: appdb_equiplink.partscallcentre_parseddocumentnumber_test;;
'UnresolvedRelation [appdb_equiplink, partscallcentre_parseddocumentnumber_test]


During handling of the above exception, another exception occurred:

 Exception Traceback (most recent call last)
 <command-351079025198108> in <module> 
 ----> 1 main ( ) 

 <command-842702219804308> in main () 
 9 except Exception as e : 
 10 traceback . print_exc ( ) 
 ---> 11 raise Exception ( "Error while reading from Delta Table." ) 
 12 
 13 # Attributing revenue to calls 

 Exception : Error while reading from Delta Table.